# Information Theory and the Game Wordle (Part 1 of 2)

## Goal of this lecture (Part 1 of 2):

This lecture will be split up into 2 parts. This first part will introduce the game wordle and the basic concepts of Information Theory that we will use. All the code written in this part will be in Julia. The 2nd part of this lecture will take our knowledge of Information Theory and combine it with the game Wordle to create an algorithm that would give us an optimal guess of words given the current information.

## What is Wordle?

If you don't already know what the game Wordle is, you might be living under a rock. However, if that is true, you don't have to worry. That is because Wordle is a simple word game that operates off of a couple of rules.

In this game, you'll have 6 guesses to find the hidden 5 letter word. For every letter of each guess, you will be told either:

1. The letter does not exist in the word, marked with a gray box
2. The letter exists in the word but is not in the right spot, marked with a yellow box
3. The letter exists in the word and is in the right spot, marked with a green box

**Here is an example:**
Our first guess was "CRATE". Since the letters "CRAT" are in gray boxes, we know that none of those letters exist in this word. However, the last E is in a green box! So we know we got one! 

Our next guess is "DRIED".  Since the first D is now in a green box, we know the word begins with a D! Since there is another D in a yellow box, we know that there is at least another D somewhere in the word! 



Our final guess is the word "DODGE", which shows up in all green which means that we got the word in just 3 guesses!\
![Alt text](wordleExample.png)

## What is our goal?

The goal of the game Wordle is to find the correct word within 6 guesses. However, for most players, we enjoy finding the word in the least amount of guesses possible! So, we want to be able to create an algorithm that would help us achieve this!


One thing that we could try to do is to remove as many possibilities per guess as possible! In a simple guessing game, like trying to guess a number between 1 and 100 and being told if we are high or low! We can see that the least optimal way of guessing would be in order. For example, if the number is 75, then it would take 75 guesses to get the right answer! The optimal way would be to cut the numbers in half each time! Watch how this works:
1. 50 (low, so well go into the middle of the higher interval)
2. 75 (correct!)
We were able to reduce our guess from 75 to just 2 using this algorithm! In fact, using this algorithm, the highest amount of guesses required would be $\log_2(n)$ where n is the number of possible guesses. So that would mean that most amount of guesses for our number game is just 7!

This is actually the equivalent of a binary search on a list.

In [12]:
# Recursive example of this binary search
function numberGuesser(rangeStart, rangeEnd, target)
    if rangeStart > rangeEnd
        return
    end
        
    currentGuess = div((rangeStart + rangeEnd), 2) # Sets currentGuess to middle of the range
    println(currentGuess)
        
    if currentGuess == target
        return
    elseif currentGuess > target
        numberGuesser(rangeStart, currentGuess - 1, target)
    else
        numberGuesser(currentGuess + 1, rangeEnd, target)
    end
end

for i in 1:5
    println(string(2*i^2, ": "))
    numberGuesser(1, 100, 2*i^2)
    println()
end

2: 
50
25
12
6
3
1
2

8: 
50
25
12
6
9
7
8

18: 
50
25
12
18

32: 
50
25
37
31
34
32

50: 
50



This is great, but we can't exactly do this with Wordle. With our brains, we could usually guess things that would give us more information, but it is hard to think of what guesses would cut the amount of possible guesses down the most would be!


Here is where Information Theory will come in handy!

## Basics of Information Theory

In Information Theory, the unit of information used is called **the bit**. The amount of bits is usually represented with the letter I (for information) and can be solved with this equation $I = -\log_2(p)$ where p how much smaller the space of possibilities becomes. Here is an example:

In [2]:
# This Julia file includes all the words that Wordle will accept as a guess, as well as all the possible answers.
include("words.jl") # I stole this information straight from the Wordle source code. 

length(words) + length(answers) # The words and answers are disjoint but you can type all of them in

12972

In [9]:
-log(670/10657)/log(2) # Number of bits of information from our example

3.991496463421989

There are 12972 possible guesses that Wordle will accept. In reality, there are less words that it would use as answers, but let's ignore that. Now, let's say that the word CRATE cuts the number of guesses down from 12972 to just 810, then the amount of guesses have been cut down by a factor of almost 16! That means that amount of information that the word CRATE gives us is about $I = -\log_2(1/16) = 4$! By finding the word for each guess that gives us the most amount of information or bits, we can reduce the number of possible guesses by as much as possible per guess!

So how can we find which word would give us the most amount of information?

## Entropy!

There is a way for us to calculate the expected information that a word could give us! This function looks like this $E(w) = \displaystyle\sum_{x}-p(x)\log_2(p(x))$. This is called the **Entropy Function**.\
Here, $w$ is the word that we are guessing. $x$ is one of the possible results that could appear on the screen, and $p(x)$ is the probability of each of those results.

For example, if the word is "CRATE", the first x could be the result where all letters are gray. Then, $p(x$) is the probability of getting that result, and $-log_2(x)$ is the amount of information of that result. When we take the sum, we get the expected information of each guess.

If we had a uniform distribution of possibilities, the Entropy function would be equal to $\log_2(3^5)$ since there are $3^5$ equally likely outcomes.

In [3]:
log(3^5)/log(2)

7.924812503605781

The result of that is about 8. Which means that, at most, we would be able to cut our set down in half about 8 times! So there is a strict upperbound for our expected entropy value. However, since the probability of each outcome i not uniform, we can expect our entropy values to be lower than that for all of our guesses.

So now, we want to find the word with the highest entropy, or the highest expected information!

## Conclusion for part 1!

Now that you understand how the game Wordle works, and the basics of Information Theory, we can now write a program that can find us the optimal guess!